<h1 style="color:rgb(0,120,170)">Neural Networks and Deep Learning</h1>
<h2 style="color:rgb(0,120,170)">Intro to Tensor Flow and Keras</h2>
<h2 style="color:rgb(0,120,170)">Testing model hiperparameters</h2>


!pip install -U tensorflow tensorboard

In [1]:
import os
import sys
#import re
import time
import datetime
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
%matplotlib inline

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorboard.plugins.hparams import api_pb2
from tensorboard.plugins.hparams import summary as hparams_summary
%load_ext tensorboard

from google.protobuf import struct_pb2

In [2]:
print(tf.__version__)

2.3.1


In [3]:
print(os.listdir("../../../data"))

# Clear any logs from previous runs
#!rm -rf ./logs/ 

['.ipynb_checkpoints', 'airbnb.csv', 'best_model.h5', 'best_parameters_svm.pickle', 'Concrete_Data.xlsx', 'gender_submission.csv', 'ion.csv', 'kaggle_digits_sample_submission.csv', 'kaggle_digits_test.zip', 'kaggle_digits_train.zip', 'kaggle_titanic_test.csv', 'kaggle_titanic_train.csv', 'London_accidents_merged.csv', 'MNIST', 'pima-indians-diabetes.csv', 'titanic.jpeg', 'tweets.csv.gz', 'UK_LSOA_bounding_boxes.csv', 'winequality-red.csv']


## 1st - [Concrete data](https://medium.com/@rahul.raoniar/hyperparameter-tuning-of-keras-deep-learning-model-in-python-e426c94a43d7)

In [4]:
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

from sklearn import preprocessing
from sklearn.model_selection import RandomizedSearchCV, KFold, cross_val_score

In [5]:
concrete = pd.read_excel('../../../Data/Concrete_Data.xlsx')
concrete.head()

,Cement,BF_slag,Fly_ash,Water,Super_plasticizer,Coarse_aggregate,Fine_aggregate,Age,Comp_str
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


In [6]:
concrete.shape

(1030, 9)

In [7]:
train_inputs = concrete.drop("Comp_str", axis = 1).values
train_targets = concrete["Comp_str"].values

## Data Scaling 

In [8]:
train_inputs =  preprocessing.scale(train_inputs)

In [9]:
print(train_inputs.shape)
print(train_targets.shape)

(1030, 8)
(1030,)


In [10]:
n_cols = train_inputs.shape[1]
input_shape = (n_cols, )

# Creates a model given an activation and learning rate
def create_model(learning_rate=0.01, activation='relu'):
  
    # Create an Adam optimizer with the given learning rate
    opt = Adam(lr=learning_rate)
  
    # Create your binary classification model  
    model = Sequential()
    model.add(Dense(128, activation = activation,
                         input_shape = input_shape,
                         activity_regularizer=regularizers.l2(1e-5)))
    model.add(Dropout(0.50))
    model.add(Dense(128, activation = activation, 
                         activity_regularizer=regularizers.l2(1e-5)))
    model.add(Dropout(0.50))
    model.add(Dense(1, activation = activation))

# Compile the model
    model.compile(optimizer = opt,
                  loss = "mean_absolute_error",
                  metrics=['mse', "mape"])
    return model

In [ ]:
# Create a Classifier
model = KerasRegressor(build_fn = create_model,
                       verbose = 0)

# Define the parameters to try out
params = {'activation': ["relu", "tanh"],
          'batch_size': [16, 32, 64], 
          'epochs': [50, 100],
          'learning_rate': [0.01, 0.001, 0.0001]}

# Create a randomize search cv object passing in the parameters to try
random_search = RandomizedSearchCV(model,
                                   param_distributions = params,
                                   cv = KFold(10))

random_search_results = random_search.fit(train_inputs, train_targets)

## Best parameters

In [ ]:
# Print best score and parameters
print("Best Score: ", random_search_results.best_score_, "and Best Params: ", random_search_results.best_params_)

# Fitting model with best parameters

In [ ]:
n_cols = train_inputs.shape[1]
input_shape = (n_cols, )

# Creates a model given an activation and learning rate
def create_model(learning_rate=0.001, activation='relu'):
  
    # Create an Adam optimizer with the given learning rate
    opt = Adam(lr=learning_rate)
  
    # Create your binary classification model  
    model = Sequential()
    model.add(Dense(128, activation = activation,
                         input_shape = input_shape,
                         activity_regularizer=regularizers.l2(1e-5)))
    model.add(Dropout(0.50))
    model.add(Dense(128, activation = activation, 
                         activity_regularizer=regularizers.l2(1e-5)))
    model.add(Dropout(0.50))
    model.add(Dense(1, activation = activation))

# Compile the model
    model.compile(optimizer = opt,
                  loss = "mean_absolute_error",
                  metrics= ['mse', "mape"])
    return model

In [ ]:
# Create a KerasClassifier
model = KerasRegressor(build_fn = create_model,
                       epochs = 100, 
                       batch_size = 16,
                       verbose = 0)

# Calculate the accuracy score for each fold
kfolds = cross_val_score(model,
                         train_inputs,
                         train_targets,
                         cv = 10)

# Print the mean accuracy
print('The mean accuracy was:', kfolds.mean())

# Print the accuracy standard deviation
print('With a standard deviation of:', kfolds.std())

## 2nd - Titanic data

In [ ]:
train_data = pd.read_csv("../../../data/kaggle_titanic_train.csv")
test_data = pd.read_csv("../../../data/kaggle_titanic_test.csv")

train_data.info()
print('_'*50)
test_data.info()

In [ ]:
train_data.head()

In [ ]:
train_data.describe()

#### Drop the columns if the missing values are more than 20% of the data. Drop missing rows otherwise

In [ ]:
for col in train_data.columns:
    if len(train_data[col].dropna()) <= (0.7 * len(train_data)):
        train_data.drop(columns=[col], inplace=True)
    else:
        train_data.dropna(axis=0, subset=[col],inplace=True)

for col in test_data.columns:
    if len(test_data[col].dropna()) <= (0.7 * len(test_data)):
        test_data.drop(columns=[col], inplace=True)
    else:
        test_data[col].fillna(value=test_data[col].mode()[0] ,inplace=True)

In [ ]:
train_data.info()
print('_'*50)
test_data.info()

No need to convert the data types from categorical to numerical because we want tensorflow to handle this kind of data

#### Define Feature columns for tensorflow  
Examples of each column type

In [ ]:
# Just to see the correlation
plt.figure(figsize=(10,8))
sns.heatmap(train_data.corr(method='pearson'), annot=True, cmap='YlGnBu', fmt='.2f', linewidths=2)

In [ ]:
feature_columns = []

# numeric cols
for header in ['Age', 'Fare']:
    feature_columns.append(tf.feature_column.numeric_column(header))

# bucketized cols
age = tf.feature_column.numeric_column("Age")
age_buckets = tf.feature_column.bucketized_column(age, boundaries=[5, 10, 20, 30, 40, 50, 60, 70, 80])
feature_columns.append(age_buckets)

# indicator cols
categorical_cols = ["Sex", "Embarked", "Pclass", "SibSp", "Parch"]
for col in categorical_cols:
    train_data[col] = train_data[col].apply(str)
    test_data[col] = test_data[col].apply(str)
    cat_column_with_vocab = tf.feature_column.categorical_column_with_vocabulary_list(col, list(train_data[col].value_counts().index.values))
    one_hot = tf.feature_column.indicator_column(cat_column_with_vocab)
    feature_columns.append(one_hot)


# embedding cols
ticket = tf.feature_column.categorical_column_with_hash_bucket("Ticket", hash_bucket_size=1000)
ticket_embedding = tf.feature_column.embedding_column(ticket, dimension=8)
feature_columns.append(ticket_embedding)

# crossed cols
p_class = tf.feature_column.categorical_column_with_vocabulary_list("Pclass", list(train_data["Pclass"].value_counts().index.values))
parch = tf.feature_column.categorical_column_with_vocabulary_list("Parch", list(train_data["Parch"].value_counts().index.values))
pclass_parch_crossed = tf.feature_column.crossed_column([p_class, parch], hash_bucket_size=1000)
pclass_parch_crossed = tf.feature_column.indicator_column(pclass_parch_crossed)
feature_columns.append(pclass_parch_crossed)

In [ ]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, testing=False, batch_size=32):
    dataframe = dataframe.copy()
    if not testing:
        labels = dataframe.pop('Survived')
        ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
        ds = ds.shuffle(buffer_size=len(dataframe))
    else:
        ds = tf.data.Dataset.from_tensor_slices(dict(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return ds

In [ ]:
train_data, val_data = train_test_split(train_data, test_size=0.2)

In [ ]:
print(train_data.shape)
print(val_data.shape)
print(test_data.shape)

In [ ]:
batch_size = 32
train_ds = df_to_dataset(train_data, batch_size=batch_size)
val_ds = df_to_dataset(val_data, batch_size=batch_size)
test_ds = df_to_dataset(test_data, testing=True, batch_size=batch_size)

### Setup Hyperparameter tuning

In [ ]:
num_units_list = [128, 256]
dropout_rate_list = [0.2, 0.5] 
optimizer_list = ['adam', 'sgd'] 

In [ ]:
# Utility method to create summary for tensorboard
def create_experiment_summary(num_units_list, dropout_rate_list, optimizer_list):
    num_units_list_val = struct_pb2.ListValue()
    num_units_list_val.extend(num_units_list)
    dropout_rate_list_val = struct_pb2.ListValue()
    dropout_rate_list_val.extend(dropout_rate_list)
    optimizer_list_val = struct_pb2.ListValue()
    optimizer_list_val.extend(optimizer_list)
    return hparams_summary.experiment_pb(hparam_infos=[api_pb2.HParamInfo(name='num_units',                              # The parameters being tracked
                                                                          display_name='Number of units',
                                                                          type=api_pb2.DATA_TYPE_FLOAT64,
                                                                          domain_discrete=num_units_list_val),
                                                       api_pb2.HParamInfo(name='dropout_rate',
                                                                          display_name='Dropout rate',
                                                                          type=api_pb2.DATA_TYPE_FLOAT64,
                                                                          domain_discrete=dropout_rate_list_val),
                                                       api_pb2.HParamInfo(name='optimizer',
                                                                          display_name='Optimizer',
                                                                          type=api_pb2.DATA_TYPE_STRING,
                                                                          domain_discrete=optimizer_list_val)], 
                                         metric_infos=[api_pb2.MetricInfo(name=api_pb2.MetricName(tag='accuracy'),        # The metrics being tracked
                                                                          display_name='Accuracy'),
                                                      ])

exp_summary = create_experiment_summary(num_units_list, dropout_rate_list, optimizer_list)
root_logdir_writer = tf.summary.create_file_writer("logs/hparam_tuning")

with root_logdir_writer.as_default():
    #tf.summary.import_event(tf.compat.v1.Event(summary=exp_summary).SerializeToString()) #https://github.com/tensorflow/tensorboard/pull/2110
    tf.summary.experimental.write_raw_pb(tf.compat.v1.Event(summary=exp_summary).SerializeToString(), step=tf.compat.v1.Event(summary=exp_summary).step)

    


In [ ]:
# Model compiler
def train_test_model(hparams):
    tf.keras.backend.set_floatx('float64')
    model = tf.keras.models.Sequential([tf.keras.layers.DenseFeatures(feature_columns),
                                        tf.keras.layers.Dense(hparams['num_units'], activation='relu'),
                                        tf.keras.layers.Dropout(hparams['dropout_rate']),
                                        tf.keras.layers.Dense(hparams['num_units'], activation='relu'),
                                        tf.keras.layers.Dense(1, activation='sigmoid')])
    
    model.compile(optimizer=hparams['optimizer'],
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    model.fit(train_ds,
              validation_data=val_ds,
              epochs=50,
              use_multiprocessing=True,
              verbose=0)
    
    _, accuracy = model.evaluate(val_ds)
    return model, accuracy

In [ ]:
# Model runner
def run(run_dir, hparams):
    writer = tf.summary.create_file_writer(run_dir)
    summary_start = hparams_summary.session_start_pb(hparams=hparams)
    with writer.as_default():
        model, accuracy = train_test_model(hparams)
        summary_end = hparams_summary.session_end_pb(api_pb2.STATUS_SUCCESS)

    tf.summary.scalar('accuracy', accuracy, step=1, description="The accuracy")
    #tf.summary.import_event(tf.compat.v1.Event(summary=summary_start).SerializeToString())
    #tf.summary.import_event(tf.compat.v1.Event(summary=summary_end).SerializeToString())
    tf.summary.experimental.write_raw_pb(tf.compat.v1.Event(summary=summary_start).SerializeToString(), step=tf.compat.v1.Event(summary=exp_summary).step)
    tf.summary.experimental.write_raw_pb(tf.compat.v1.Event(summary=summary_end).SerializeToString(), step=tf.compat.v1.Event(summary=exp_summary).step)
    return model, accuracy

In [ ]:
tf.get_logger().setLevel('WARNING')

model_dict = {}
session_num = 0
for num_units in num_units_list:
    for dropout_rate in dropout_rate_list:
        for optimizer in optimizer_list:
            hparams = {'num_units': num_units, 'dropout_rate': dropout_rate, 'optimizer': optimizer}
            print(f'--- Running training session {session_num + 1}')
            print(hparams)
            run_name = f"run-{session_num}"
            model, accuracy = run("logs/hparam_tuning/" + run_name, hparams)
            print(accuracy)
            model_dict[accuracy] = model
            session_num += 1

In [ ]:
best_model = model_dict[max(list(model_dict.keys()))]

In [ ]:
predictions = best_model.predict(test_ds)
predictions = np.argmax(predictions, axis=1)

In [ ]:
predictions_dataframe = test_data[["PassengerId"]]
predictions_dataframe["Survived"] = predictions

In [ ]:
#predictions_dataframe.to_csv("../../../data/gender_submission.csv",index=False)

In [ ]:
#best_model.save('../../../data/best_model.h5')

## 3rd - [Using Talos](https://towardsdatascience.com/hyperparameter-optimization-with-keras-b82e6364ca53) (tensorflow 1.X --- will need to observe for the future)

!pip install git+https://github.com/autonomio/talos@1.0

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense

%matplotlib inline

import talos

### 1. The Required Inputs and Data

In [ ]:
# then we load the dataset
x, y = talos.templates.datasets.breast_cancer()

# and normalize every feature to mean 0, std 1
x = talos.utils.rescale_meanzero(x)

### 2. Creating the Keras Model

In [ ]:
# first we have to make sure to input data and params into the function
def breast_cancer_model(x_train, y_train, x_val, y_val, params):

    model = Sequential()
    model.add(Dense(params['first_neuron'], input_dim=x_train.shape[1],
                    activation=params['activation'],
                    kernel_initializer=params['kernel_initializer']))
    
    model.add(Dropout(params['dropout']))

    model.add(Dense(1, activation=params['last_activation'],
                    kernel_initializer=params['kernel_initializer']))
    
    model.compile(loss=params['losses'],
                  optimizer=params['optimizer'],
                  metrics=['acc', talos.utils.metrics.f1score])
    
    history = model.fit(x_train, y_train, 
                        validation_data=[x_val, y_val],
                        batch_size=params['batch_size'],
                        callbacks=[talos.utils.live()],
                        epochs=params['epochs'],
                        verbose=0)

    return history, model

### 3. Defining the Parameter Space Boundary

In [ ]:
# then we can go ahead and set the parameter space
p = {'first_neuron':[9,10,11],
     'hidden_layers':[0, 1, 2],
     'batch_size': [30],
     'epochs': [100],
     'dropout': [0],
     'kernel_initializer': ['uniform','normal'],
     'optimizer': ['Nadam', 'Adam'],
     'losses': ['binary_crossentropy'],
     'activation':['relu', 'elu'],
     'last_activation': ['sigmoid']}

### 4. Starting the Experiment

In [ ]:
# and run the experiment
t = talos.Scan(x=x,
               y=y,
               model=breast_cancer_model,
               params=p,
               experiment_name='breast_cancer',
               round_limit=10
              )